#Introduction

Title: Exam MCQ Generator

#Installs

In [1]:
!pip install -q python-dotenv

!pip install -q langchain_experimental
!pip install -q langchain

!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requ

In [2]:
!pip install -q pydantic

#Imports

In [3]:
import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

#Data Source

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
def read_text_file(path):
  with open(path, 'r', encoding='utf-8') as f:
    text = f.read()

  return text

In [7]:
texts_path = '/content/drive/MyDrive/datasets/exam_mcq_generator/texts'

text_path_01 = os.path.join(texts_path, '01.txt')
text_01 = read_text_file(text_path_01)

text_path_02 = os.path.join(texts_path, '02.txt')
text_02 = read_text_file(text_path_02)

text_path_03 = os.path.join(texts_path, '03.txt')
text_03 = read_text_file(text_path_03)

text_path_04 = os.path.join(texts_path, '04.txt')
text_04 = read_text_file(text_path_04)

text_path_05 = os.path.join(texts_path, '05.txt')
text_05 = read_text_file(text_path_05)

#Env

In [8]:
env_path = '/content/drive/MyDrive/credentials/data-analytics-demo/.env'

In [9]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(env_path)

True

In [10]:
GOOGLE_PALM_API_KEY = os.environ['GOOGLE_PALM_API_KEY']
HUGGINGFACE_API_KEY = os.environ['HUGGINGFACE_API_KEY']
OPEN_AI_API_KEY = os.environ['OPEN_AI_API_KEY']

# GOOGLE_PALM_API_KEY, HUGGINGFACE_API_KEY, OPEN_AI_API_KEY

#Model

In [14]:
from pydantic import BaseModel, Field

class MCQModel(BaseModel):
  question: str = Field(description="This is the question text")
  options: list[str] = Field(description="This is a list of multiple choices or options avalible")
  correct_option: str = Field(description="This is the correct choice or option")

In [15]:
class MCQListModel(BaseModel):
  mcq_list: list[MCQModel] = Field(description="A list of multiple choice questions")

In [16]:
from langchain.output_parsers import PydanticOutputParser

pydantic_parser = PydanticOutputParser(pydantic_object=MCQListModel)

format_instructions = pydantic_parser.get_format_instructions()

print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"mcq_list": {"title": "Mcq List", "description": "A list of multiple choice questions", "type": "array", "items": {"$ref": "#/definitions/MCQModel"}}}, "required": ["mcq_list"], "definitions": {"MCQModel": {"title": "MCQModel", "type": "object", "properties": {"question": {"title": "Question", "description": "This is the question text", "type": "string"}, "options": {"title": "Options", "description": "This is a list of multiple choices or options avalible", "type": "array", "items": {"type": "string"}}, "correct_option": {"tit

In [17]:
langchain_llm = ChatOpenAI(openai_api_key=OPEN_AI_API_KEY, model_name="gpt-3.5-turbo-0613", verbose=False,)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [18]:
prompt = (
    "Following are blocks of text picked at random from a document:"
    "\n{texts}"
    "\nGenerate {n} non-repeating multiple choice questions along with their respective correct options from this text"
    "\nDo not use any outside information"
    "\n{format_instructions}"
)

In [19]:
prompt_template = ChatPromptTemplate.from_template(prompt)

prompt_template

ChatPromptTemplate(input_variables=['format_instructions', 'n', 'texts'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'n', 'texts'], template='Following are blocks of text picked at random from a document:\n{texts}\nGenerate {n} non-repeating multiple choice questions along with their respective correct options from this text\nDo not use any outside information\n{format_instructions}'))])

In [20]:
llm_chain = LLMChain(llm=langchain_llm, prompt=prompt_template, output_key="result")

In [112]:
def get_docs(text, chunk_size=500, chunk_overlap=0):
  """
  note:
    * When chunk_overlap=0, it will try not cut sentences, which is good for out case
  """
  text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=chunk_size,
      chunk_overlap=chunk_overlap,
      length_function=len,
      is_separator_regex=False,
  )

  docs = text_splitter.create_documents([text])

  return docs

In [113]:
from langchain.callbacks import get_openai_callback as get_langchain_openai_callback

import random

def get_exam(text, n=3):
  """
  validation: total_docs <= n
  """

  docs = get_docs(text)

  if n > len(docs):
    raise ValueError(f"Too many questions specified, must be less then {len(docs)+1}")

  rand_docs = random.sample(docs, n)

  texts = "".join([f"Text Block {i+1}:\n{doc.page_content}\n" for i, doc in enumerate(rand_docs)])

  with get_langchain_openai_callback() as cb_langchain:
    response = llm_chain.invoke({"texts": texts, "n": n, "format_instructions": format_instructions})

  return {"mcqs_parsed": MCQListModel.parse_raw(response['result']).mcq_list, 'cb_langchain': cb_langchain}

# get_exam(text_01)

#Testing

In [100]:
def display_exam(exam):

  for i in exam['mcqs_parsed']:

    print(f"question: {i.question}\n")

    for index, j in enumerate(i.options):
      print(f"{index+1}. {j}")

    print(f"\ncorrect: {i.correct_option}\n\n")

  print(exam['cb_langchain'])

In [117]:
exam_01 = get_exam(text_01, n=10)

ValueError: Too many questions specified, must be less then 8

In [102]:
exam_01 = get_exam(text_01, n=5)

In [103]:
display_exam(exam_01)

question: What did Seraphina discover about the archipelago?

1. It was home to mythical creatures
2. It held the key to a realm between reality and dreams
3. It was a place of lost civilizations
4. It had boundaries of time and space intertwined

correct: It held the key to a realm between reality and dreams


question: What was the undercurrent that stirred within Seraphina's soul?

1. Restlessness
2. Adventure
3. Curiosity
4. Uncertainty

correct: Restlessness


question: What did Seraphina find half-buried in the sand?

1. An ancient artifact
2. A glowing stone
3. A treasure chest
4. A map

correct: An ancient artifact


question: What did Seraphina embark on a journey to unravel?

1. The mysteries hidden within the ancient map
2. The secrets of the archipelago
3. Her own destiny
4. The enigmatic landmarks

correct: The mysteries hidden within the ancient map


question: What did Seraphina become after her journey?

1. A curious villager
2. A guardian of realms
3. A legendary figur

In [104]:
exam_02 = get_exam(text_02, n=5)

In [105]:
display_exam(exam_02)

question: What were some of the ethical considerations surrounding SynthEra?

1. Privacy and security
2. Cost and availability
3. Physical limitations
4. Environmental impact

correct: Privacy and security


question: In which sectors did SynthEra have a significant impact?

1. Agriculture and construction
2. Medicine, education, and entertainment
3. Finance and law
4. Transportation and logistics

correct: Medicine, education, and entertainment


question: What was one of the key abilities provided by SynthEra?

1. Telepathic communication
2. Time travel
3. Instantaneous teleportation
4. Interacting with augmented reality

correct: Interacting with augmented reality


question: How did SynthEra establish a connection between the human brain and digital information?

1. Through physical wires
2. By using quantum technology
3. Via advanced neurostimulation algorithms
4. By implanting microchips in the brain

correct: Via advanced neurostimulation algorithms


question: What did SynthEra

In [106]:
exam_03 = get_exam(text_03, n=5)

In [107]:
display_exam(exam_03)

question: What is the Collatz conjecture?

1. A puzzle that has defied resolution for decades
2. A mathematical theorem proven by Lothar Collatz
3. A sequence that eventually reaches the cycle of 4, 2, 1
4. A problem in number theory
5. A puzzle simple enough for amateurs to grasp

correct: A sequence that eventually reaches the cycle of 4, 2, 1


question: Who proposed the Collatz conjecture?

1. A group of scholars and amateurs
2. Lothar Collatz
3. German mathematicians
4. Mathematicians exploring the unknown
5. The mid-20th century intellectuals

correct: Lothar Collatz


question: What does the resolution of the Collatz conjecture promise?

1. A solution to a mathematical puzzle
2. A deeper understanding of the beauty of numbers
3. A testament to the allure of the unknown
4. A boundless exploration of infinite realms
5. All of the above

correct: All of the above


question: What is the allure of the Collatz conjecture?

1. Its simplicity that belies its depth
2. The mysteries with

In [108]:
exam_04 = get_exam(text_04, n=5)

In [109]:
display_exam(exam_04)

question: What is the aesthetic allure of poetry?

1. Its ability to distill intricate emotions into a few carefully chosen words
2. Its capacity to evoke profound verities
3. Its manifestation in sundry configurations and idioms
4. Its nuance in syllabic configurations and rhythmic resonances

correct: Its ability to distill intricate emotions into a few carefully chosen words


question: What does poetry constitute?

1. A medium for the poet's oration
2. A testament to the boundless potentiality of words
3. A tapestry of allegorical impressions
4. A convolution of lexical cadence

correct: A testament to the boundless potentiality of words


question: What does poetry endure as?

1. An enigmatic manifestation
2. An everlasting testimonial
3. A contrivance of language
4. A convolution of lexical cadence

correct: An everlasting testimonial


question: What does the nuanced choreography of syllabic configurations and rhythmic resonances enable poets to do?

1. Forge connections betwixt

In [110]:
exam_05 = get_exam(text_05, n=5)

In [111]:
display_exam(exam_05)

question: What are researchers currently focusing on in LLMs?

1. Refining the models and addressing ethical concerns
2. Developing rule-based systems
3. Utilizing transformer architectures
4. Translating Russian sentences into English

correct: Refining the models and addressing ethical concerns


question: What were the limitations of early rule-based systems in natural language processing?

1. They struggled with the ambiguity and complexity of language
2. They were constrained by limited computational power
3. They lacked the ability to consider contextual information
4. They failed to achieve true linguistic nuance

correct: They struggled with the ambiguity and complexity of language


question: Which model revolutionized natural language processing?

1. LLMs
2. Rule-based systems
3. Transformer architectures
4. Georgetown-IBM experiment

correct: Transformer architectures


question: What was the outcome of the Georgetown-IBM experiment in 1954?

1. Successful translation of Rus